In [1]:
import os
import pandas as pd
from NS_eval_utils import load_model_tokenizer, load_RT_data, extract_stories_from_df, tokenize_story, multitoken_wordmap, get_model_surprisals

In [32]:
#dataset = "babylm_full_bpe_8K"

tokenizers_root = r"/home/abishekthamma/PycharmProjects/masters_thesis/ss-llm/nanoGPT/data"
out_root = r'/home/abishekthamma/PycharmProjects/masters_thesis/ss-llm/nanoGPT/output_dump'

# data_folder = r'babylm_full_bpe'
# model_name = 'out-babylm_full_bpe-6x6-nomask-5492134'

# data_folder = r'babylm_full_bpe_8k'
# model_name = r'out-babylm_full_bpe_8k-2x2-mask_e010-5983308'

data_folder = 'babylm_wocdes_full_bpe'
model_name = 'out-babylm_wocdes_full_bpe-4x4-nomask-5445338'


#model_name = model_list[0]

out_dir = os.path.join(out_root, model_name)
data_dir = os.path.join(tokenizers_root, data_folder)


In [33]:
# Load the model and tokenizer
model, tokenizer = load_model_tokenizer(out_dir=out_dir, data_dir=data_dir)
processed_RTs, processed_wordinfo, all_stories = load_RT_data(rt_root=r'naturalstories_RTS')

Loading model from /home/abishekthamma/PycharmProjects/masters_thesis/ss-llm/nanoGPT/output_dump/out-babylm_wocdes_full_bpe-4x4-nomask-5445338/ckpt.pt
number of parameters: 7.24M
Loading custom tokenizer from /home/abishekthamma/PycharmProjects/masters_thesis/ss-llm/nanoGPT/data/babylm_wocdes_full_bpe


In [34]:
all_stories.groupby('item').count()['zone'].reset_index().rename(columns={'zone':'wordcount'})

,item,wordcount
0,1,1073
1,2,990
2,3,1040
3,4,1085
4,5,1013
5,6,1099
6,7,999
7,8,980
8,9,1038
9,10,939


"item" - Story ID 

"zone" - word number in the story

in processed_RTs -
 
"WorkerId" is the worker ID |
"RT" - response time


in processed_wordinfo -

nItem - Number of people have datapoint for that word in that story (???)



In [35]:
#Finding minimum number of people who read all the words in the story
processed_wordinfo.groupby('item')['nItem'].min()

item
1     77
2     79
3     87
4     85
5     80
6     78
7     71
8     68
9     67
10    62
Name: nItem, dtype: int64

In [36]:
#Filter for WorkerIds where count of WorkerIds for an item is equal to wordcount

#Basically merge processed_RTs which is at worker, story, word level with all_stories grouped by story and counted for zones which is words. To get wordcount for each story. Now use this to count wordcounts (number of rows) (how many words of a story a particular worker has read) by grouping by workerid and item and counting for zones and get worker wordcount. Now merge this with previous merge and then query for WorkerId_count == wordcount to get the workers who have read all the words in the story

processed_RTs.merge(all_stories.groupby('item').count()["zone"].reset_index().rename(columns={'zone':'wordcount'}),on='item').merge(processed_RTs[["item", "WorkerId","zone"]].groupby(["item", "WorkerId"]).count().reset_index().rename(columns={"zone":"WorkerId_count"}), on=["item", "WorkerId"]).query("WorkerId_count == wordcount")

,WorkerId,WorkTimeInSeconds,correct,item,zone,RT,word,nItem,meanItemRT,sdItemRT,gmeanItemRT,gsdItemRT,wordcount,WorkerId_count
0,A117RW2F1MNBQ8,2792,6,1,1,360,If,84,369.011905,160.579935,340.566023,1.490513,1073,1073
1,A117RW2F1MNBQ8,2792,6,1,2,304,you,87,368.183908,168.027166,339.746188,1.475379,1073,1073
2,A117RW2F1MNBQ8,2792,6,1,3,270,were,88,344.318182,224.916666,310.337960,1.505613,1073,1073
3,A117RW2F1MNBQ8,2792,6,1,4,292,to,86,354.639535,310.065644,305.686701,1.603190,1073,1073
4,A117RW2F1MNBQ8,2792,6,1,5,304,journey,89,349.674157,198.212855,315.986066,1.530090,1073,1073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844901,AMYL22GHCON9A,2308,6,10,935,312,and,70,317.414286,112.983581,300.670987,1.380108,939,939
844902,AMYL22GHCON9A,2308,6,10,936,296,better,71,335.267606,134.405353,312.120415,1.453014,939,939
844903,AMYL22GHCON9A,2308,6,10,937,376,treatments,72,359.861111,107.835595,342.418649,1.388847,939,939
844904,AMYL22GHCON9A,2308,6,10,938,352,for,70,506.728571,339.573770,437.976093,1.664676,939,939


In [37]:
#Filter for WorkerIds that have read all 10 stories completely

processed_RTs.merge(all_stories.groupby('item').count()["zone"].reset_index().rename(columns={'zone':'wordcount'}),on='item').merge(processed_RTs[["item", "WorkerId","zone"]].groupby(["item", "WorkerId"]).count().reset_index().rename(columns={"zone":"WorkerId_count"}), on=["item", "WorkerId"]).query("WorkerId_count == wordcount")[["WorkerId", "item"]].groupby(["item", "WorkerId"]).count().reset_index().groupby("WorkerId").count().sort_values("item", ascending=False)

#Looks like its only 1 worker who has read all 10 stories completely

,item
WorkerId,
A2U750ON775R3L,10
A2MVFSJBKHUPH9,6
A3S3YW6II31OZV,6
AEUGCBFJPU0MX,6
A2GNS1SQLQNMR2,5
...,...
A5E076UK5CA9J,1
A279TEENVAAZNB,1
A26LA7PUZG0N8S,1


In [38]:
#Same but filter for WorkerIds that have read x percentage of the stories completely
x = 0.95
processed_RTs.merge(all_stories.groupby('item').count()["zone"].reset_index().rename(columns={'zone':'wordcount'}),on='item').merge(processed_RTs[["item", "WorkerId","zone"]].groupby(["item", "WorkerId"]).count().reset_index().rename(columns={"zone":"WorkerId_count"}), on=["item", "WorkerId"]).assign(percentage = lambda x: x["WorkerId_count"]/x["wordcount"]).query("percentage >= @x")[["WorkerId", "item"]].groupby(["item", "WorkerId"]).count().reset_index().groupby("WorkerId").count().sort_values("item", ascending=False)

#Remove last groupby size  to get actual workerIds

,item
WorkerId,
A3S3YW6II31OZV,10
A2TJHKHM8T2SV3,10
A2U750ON775R3L,10
ACUXGYJAQUZAE,10
A2MVFSJBKHUPH9,10
...,...
A3ALSJWTGWPVN8,1
A12QOWM20FCN2Q,1
AO8HNJRGZQD0A,1


###### What is the use to look at individual workerers who've read all the stories. Aren't stories independent of each other?

In [23]:
t1 = processed_RTs.merge(all_stories.groupby('item').count()["zone"].reset_index().rename(columns={'zone':'wordcount'}),on='item').merge(processed_RTs[["item", "WorkerId","zone"]].groupby(["item", "WorkerId"]).count().reset_index().rename(columns={"zone":"WorkerId_count"}), on=["item", "WorkerId"])
import matplotlib.pyplot as plt
plt.plot([t1.assign(percentage = lambda x: x["WorkerId_count"]/x["wordcount"]).query("percentage >= @y")[["WorkerId", "item"]].groupby(["item", "WorkerId"]).count().reset_index().groupby("WorkerId").count().sort_values("item", ascending=False).groupby("item").size().sum() for y in map(lambda i: i/100.0, range(1, 100, 1))], label="Number of workers who've read x% of atleast 1 story")
plt.title("Number of workers who've read x% of atleast 1 story")

KeyboardInterrupt: 

In [39]:
out_dir = os.path.join(out_root, model_name)
data_dir = os.path.join(tokenizers_root, data_folder)

model, tokenizer = load_model_tokenizer(out_dir, data_dir)
processed_RTs, processed_wordinfo, all_stories = load_RT_data(rt_root=r'naturalstories_RTS')
stories = extract_stories_from_df(all_stories)

token_map_df = pd.DataFrame()
for i, story in stories.items():
    tokenized_story_i = tokenize_story(story, tokenizer)
    token_map = multitoken_wordmap(tokenized_story_i, i, processed_wordinfo, tokenizer)
    token_map_df = pd.concat([token_map_df, pd.DataFrame(token_map)])

story_surprisals_df = pd.DataFrame()
for i, story in stories.items():
    #print(f"Processing story {i}")
    story_surprisals = get_model_surprisals(model, token_map_df, i, tokenizer)
    story_surprisals_df = pd.concat([story_surprisals_df, pd.DataFrame(story_surprisals)])



Loading model from /home/abishekthamma/PycharmProjects/masters_thesis/ss-llm/nanoGPT/output_dump/out-babylm_wocdes_full_bpe-4x4-nomask-5445338/ckpt.pt
number of parameters: 7.24M
Loading custom tokenizer from /home/abishekthamma/PycharmProjects/masters_thesis/ss-llm/nanoGPT/data/babylm_wocdes_full_bpe


In [40]:

#Add model ID??? 
#Add output directory?? 

#Don't need to store item, zone, word, tokens repeatedly. Create a key for it and store it once.
#But depending on the tokenization, they could be different. So store tokenizer 

In [41]:
story_surprisal_keys = story_surprisals_df[['item', 'zone', 'word', "tokens"]].reset_index(drop=True).reset_index()
story_surprisal_keys.rename(columns={"index":"storyword_UID"}, inplace=True)
story_surprisal_keys["tokenizer"] = data_folder
story_surprisal_keys


,storyword_UID,item,zone,word,tokens,tokenizer
0,0,1,1,If,[331],babylm_wocdes_full_bpe
1,1,1,2,you,[210],babylm_wocdes_full_bpe
2,2,1,3,were,[390],babylm_wocdes_full_bpe
3,3,1,4,to,[196],babylm_wocdes_full_bpe
4,4,1,5,journey,[4751],babylm_wocdes_full_bpe
...,...,...,...,...,...,...
10251,10251,10,935,and,[207],babylm_wocdes_full_bpe
10252,10252,10,936,better,[991],babylm_wocdes_full_bpe
10253,10253,10,937,treatments,"[2100, 1284]",babylm_wocdes_full_bpe
10254,10254,10,938,for,[249],babylm_wocdes_full_bpe


In [42]:
read_story_surprisal_keys = pd.read_csv("story_surprisal_keys.csv")
max_uid = read_story_surprisal_keys["storyword_UID"].max()
story_surprisal_keys["storyword_UID"] = story_surprisal_keys["storyword_UID"] + max_uid + 1
read_story_surprisal_keys = pd.concat([read_story_surprisal_keys, story_surprisal_keys])

#read_story_surprisal_keys.to_csv("story_surprisal_keys.csv", index=False)
#story_surprisal_keys.to_csv("story_surprisal_keys.csv", index=False)

In [43]:
read_story_surprisal_keys.sort_values(["item", "zone"])

,storyword_UID,item,zone,word,tokens,tokenizer
0,0,1,1,If,[333],babylm_full_bpe_8k
10256,10256,1,1,If,[333],babylm_full_bpe
0,20512,1,1,If,[331],babylm_wocdes_full_bpe
1,1,1,2,you,[208],babylm_full_bpe_8k
10257,10257,1,2,you,[208],babylm_full_bpe
...,...,...,...,...,...,...
20510,20510,10,938,for,[248],babylm_full_bpe
10254,30766,10,938,for,[249],babylm_wocdes_full_bpe
10255,10255,10,939,Tourette's.,"[1912, 1111, 482, 225, 14]",babylm_full_bpe_8k
20511,20511,10,939,Tourette's.,"[1912, 1111, 482, 225, 14]",babylm_full_bpe


In [44]:
read_story_surprisal_keys.to_csv("story_surprisal_keys.csv", index=False)

In [45]:
def convert_modelname_to_id(inp_model_name):
    if inp_model_name.startswith('out'):
        if "curr" in inp_model_name:
            try:
                a_id = inp_model_name.split('-')[-1]
            except Exception as e:
                print(e, inp_model_name)
                return None
        else:
            try:
                a_id = inp_model_name.split('-')[4]
            except Exception as e:
                print(e, inp_model_name)
                return None
        if a_id.isdigit():
            run_data_key = int(a_id)
        elif "s" in a_id or "nm" in a_id:
            run_data_key = int(a_id.split("_")[0])

    return run_data_key


write_df = story_surprisals_df.merge(story_surprisal_keys[["storyword_UID", "item", "zone"]], on=["item", "zone"])[["storyword_UID", "surprisal"]]
write_df["model_id"] = convert_modelname_to_id(model_name)
write_df = write_df[["model_id", "storyword_UID", "surprisal"]]
write_df.to_csv("storyword_model_surprisals.csv", index=False)


In [ ]:
model_name